# This is the code notebook

In [ ]:
#imports

Defining the scenario variables 

In [1]:
algae_initial_concentration = 0 
PET_initial_concentration = 0
MHET_initial_concentration = 0
process_duration_time = 0 #HRT

Defining the equations

In [ ]:
#microalgae concentration
    #microalgae instant variation
dxdt = algae_concentration * algae_specific_growth_rate 


#enzyme concentration
    #team chlamylicious considered 0.01 microMol/s as the enzyme production coefficient. So the enzyme concentration is determined by:
enzyme_concentration = algae_concentration * 0.01

#PET concentration
PET_V = PET_degradation_coefficient * (enzyme_concentration * PET_concentration/(km + PET_concentration))

#MHET concentration
MHET_V = MHET_degradation_coefficient * (enzyme_concentration * MHET_concentration/(km + MHET_concentration))